# answer

### import package

In [1]:
import os
import torch
import numpy as np
import math
import glob
from os import listdir
from os import walk
from torch import nn
from tqdm import tqdm 
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from torchvision import datasets, transforms
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization

import PIL.Image as Image
from matplotlib import pyplot as plt
import torch.nn.functional as F
import torchvision.models as models

### check gpu

In [2]:
data_path = '../../dataset/bird_datasets/test'
classes_path = '../../dataset/bird_datasets/classes.txt'
test_filename_path = '../../dataset/bird_datasets/testing_img_order.txt'

print('torch version:' + torch.__version__)

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Available GPUs: ', end='')
    for i in range(torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i), end=' ')
else:
    device = torch.device('cpu')
    print('CUDA is not available.')

torch version:1.8.0
Available GPUs: GeForce RTX 2080 Ti GeForce GTX 1080 Ti 

## read classes txt

In [3]:
class_to_idx = {}
with open(classes_path) as f:
    for line in f.readlines():
        label_num =  line.split(".")[0] 
        label_str =  line.split(".")[1][:-1]
        class_to_idx[int(label_num) - 1] = label_str
print(class_to_idx)

{0: 'Black_footed_Albatross', 1: 'Laysan_Albatross', 2: 'Sooty_Albatross', 3: 'Groove_billed_Ani', 4: 'Crested_Auklet', 5: 'Least_Auklet', 6: 'Parakeet_Auklet', 7: 'Rhinoceros_Auklet', 8: 'Brewer_Blackbird', 9: 'Red_winged_Blackbird', 10: 'Rusty_Blackbird', 11: 'Yellow_headed_Blackbird', 12: 'Bobolink', 13: 'Indigo_Bunting', 14: 'Lazuli_Bunting', 15: 'Painted_Bunting', 16: 'Cardinal', 17: 'Spotted_Catbird', 18: 'Gray_Catbird', 19: 'Yellow_breasted_Chat', 20: 'Eastern_Towhee', 21: 'Chuck_will_Widow', 22: 'Brandt_Cormorant', 23: 'Red_faced_Cormorant', 24: 'Pelagic_Cormorant', 25: 'Bronzed_Cowbird', 26: 'Shiny_Cowbird', 27: 'Brown_Creeper', 28: 'American_Crow', 29: 'Fish_Crow', 30: 'Black_billed_Cuckoo', 31: 'Mangrove_Cuckoo', 32: 'Yellow_billed_Cuckoo', 33: 'Gray_crowned_Rosy_Finch', 34: 'Purple_Finch', 35: 'Northern_Flicker', 36: 'Acadian_Flycatcher', 37: 'Great_Crested_Flycatcher', 38: 'Least_Flycatcher', 39: 'Olive_sided_Flycatcher', 40: 'Scissor_tailed_Flycatcher', 41: 'Vermilion_Fly

### Data augmentation

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

def get_trnsform():
    transform = transforms.Compose([
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        normalize
    ])
    return transform
trans = get_trnsform()

In [5]:
model_foloder = 'model/model_bird_resnet50_all_aug_RL'
model_idx = '0027'
with torch.no_grad():
    model = models.resnet50(pretrained=True)
    model.load_state_dict(torch.load('{}/checkpoint_{}.pth.tar'.format(model_foloder, model_idx))['state_dict'])
    model.eval().to(device)

In [12]:
def predict(path):
    img = Image.open(os.path.join(data_path, path)).convert('RGB')
    m = nn.Softmax(dim=1)
    data = trans(img).unsqueeze(0).to(device)
    y = model(data)
    temp = torch.argmax(m(y)).item() 
    label = class_to_idx[temp]
    output = str(temp + 1).zfill(3) + '.' + label
    return output

In [13]:
with open(test_filename_path) as f:
     test_images = [x.strip() for x in f.readlines()]  # all the testing images

submission = []
for img in tqdm(test_images):  # image order is important to your result
    predicted_class = predict(img)
    submission.append([img, predicted_class])

np.savetxt('answer.txt', submission, fmt='%s')

100%|██████████| 3033/3033 [01:19<00:00, 38.37it/s]
